# Create a Substation with Multiple Voltage Levels

![combined](./combined.png)

### Import CIMantic Graphs Library and CIM profile

The CIMantic Graphs library can be installed using `pip install cim-graph` and imported into any script:

In [1]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
import cimgraph.utils as utils

CIMantic Graphs can support multiple CIM profiles, so it is necessary to specify which one to use. This demonstration will use the extensions from CIMHub feature/SETO, which is derived from CIM17V38.

In [2]:
# Specify CIM Profile
import importlib
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# Connect to Blazegraph Database
from cimgraph.databases import BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)

from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
bg_loader = BlazegraphLoader(params)

To create a new model "from scratch", no filename or databased is specified. This connection object contains all information regarding the CIM version, serilaization format, etc.

In [3]:
# Create an empty connection for new file
params = ConnectionParameters(filename=None,
                              cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

For this demonstration, the Blazegraph database used by GridAPPS-D will be used for roundtrip testing. During these tests, each of the substation models developed for Grid Atlas will be added to the database. Connectivity and total load within the test feeders will be queried through a simple test.

To connect to the database, we will establish two connections. The first will be used for database bulk upload using the CIM-Loader library previously developed for Grid Atlas. The second will be used for automated database queries using CIMantic Graphs.



In [4]:
# Upload SFO model from XML file
sfo1_feeder = cim.Feeder(mRID = 'ACCDF54C-C90C-4C41-9D7A-5E5D3D17A4E5')
bg_loader.upload_from_file(filename='../test_models/p10uhs4.xml')
sfo1_network = FeederModel(connection=blazegraph, container=sfo1_feeder,
                              distributed=False)


HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 65
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="240494" milliseconds="646"/>

In [5]:
# Upload SFO model from XML file
sfo2_feeder = cim.Feeder(mRID = 'BCBCAEB9-5209-49C0-B85B-417D76618AC3')
bg_loader.upload_from_file(filename='../test_models/p1rhs5.xml')
sfo2_network = FeederModel(connection=blazegraph, container=sfo2_feeder,
                              distributed=False)


HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 63
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="15103" milliseconds="49"/>

### Create New Main-and-Transfer Substation

To create a combined substation with a main-and-transfer substation for the high-side (69kV) and a sectionalized-bus for the low-side (12.47 kV), we import the two builder classes and will then pass the `substation` object from one to the other so that both are contained in the same CIM EquipmentContainer.


In [6]:
from cimbuilder.substation_builder import MainAndTransferSubstation
from cimbuilder.substation_builder import SectionalizedBusSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation.

In [7]:
main_transfer = MainAndTransferSubstation(connection=blazegraph,
                                       name="combined_sub", 
                                       base_voltage = 69000)
substation = main_transfer.substation
network = main_transfer.network

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation. The total number of bus sections is specified by `total_sections` with a default value of two bus sections.

In [8]:
sectionalized = SectionalizedBusSubstation(connection=blazegraph,
                                        name="sectionalized_bus", 
                                        base_voltage = 12470,
                                        total_sections = 2,
                                        substation = substation,
                                        network = network)

### Create a New Transformer from Asset Catalog

To connect the high-side bus to the low-side bus, we will need to auto-generate a new CIM PowerTransformer objects and then connect it to the correct junctions within the substation. It is first necessary to create the set of breakers, airgap switches, and junctions to which the bank of two transformers will be connected:

In [9]:
high_side_1 = main_transfer.new_branch(series_number=1)
low_side_1 = sectionalized.new_branch(section_number=1, series_number=1)

high_side_2 = main_transfer.new_branch(series_number=3)
low_side_2 = sectionalized.new_branch(section_number=2, series_number=2)

To create a new piece of equipment, we import the `object_builder` module from CIM-Builder and call the `new_power_transformer` method. We will import a sample transformer template to use as a reference for creating each transformer in the bank.

In [10]:
from cimbuilder.object_builder import new_power_transformer

In [11]:
xfmr_template = '../asset_templates/power_transformer_69_12.json'

In [12]:
xfmr1 = new_power_transformer(network=main_transfer.network,
                              container=substation,
                              name='69_12_bank_1',
                              node1=high_side_1,
                              node2=low_side_1,
                              catalog_file=xfmr_template)

In [13]:
xfmr1 = new_power_transformer(network=main_transfer.network,
                              container=substation,
                              name='69_12_bank_2',
                              node1=high_side_2,
                              node2=low_side_2,
                              catalog_file=xfmr_template)

                                                

To verify that the transformers were created and added to the substation, we can print all PowerTransformer objects contained in the graph:

In [14]:
main_transfer.network.pprint(cim.PowerTransformer, use_names=True)

{
    "981c0230-ad8e-4e08-9d99-fbbea0d0aec3": {
        "mRID": "981c0230-ad8e-4e08-9d99-fbbea0d0aec3",
        "name": "69_12_bank_1",
        "EquipmentContainer": "combined_sub",
        "Terminals": "['69_12_bank_1_t1', '69_12_bank_1_t2']",
        "vectorGroup": "Yy",
        "PowerTransformerEnd": "['hvmv69_12_End_1', 'hvmv69_12_End_2']"
    },
    "60d59578-878e-4019-a182-5d05033a9a1c": {
        "mRID": "60d59578-878e-4019-a182-5d05033a9a1c",
        "name": "69_12_bank_2",
        "EquipmentContainer": "combined_sub",
        "Terminals": "['69_12_bank_2_t1', '69_12_bank_2_t2']",
        "vectorGroup": "Yy",
        "PowerTransformerEnd": "['hvmv69_12_End_1', 'hvmv69_12_End_2']"
    }
}


To add a feeder to the new substation, the `.new_feeder()` method is called. The `section_number` is used to identify the bus section to which the feeder should be connected. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model. Because the SFO synthetic feeders use a custom name for the sourcebus to be connected to the substation, that name is passed using the `sourcebus` parameter.

The function call below will automatically insert the two SFO feeders into the new substation:

In [15]:
sectionalized.new_feeder(section_number = 1, feeder=sfo1_feeder,
                        feeder_network=sfo1_network,
                        sourcebus='p10udt4214-p10uhs4_1247x')
sectionalized.new_feeder(section_number = 2, feeder=sfo2_feeder,
                        feeder_network=sfo2_network,
                        sourcebus='p1rdt1011-p1rhs5_1247x')

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [16]:
# Print substation data using names
network.pprint(cim.Substation, use_names = True)
# Print feeder data
network.pprint(cim.Feeder)

{
    "46d3dd5d-69ee-4715-a520-f659348f71b8": {
        "mRID": "46d3dd5d-69ee-4715-a520-f659348f71b8",
        "name": "combined_sub",
        "NormalEnergizedFeeder": "['feeder_p10udt4214-p10uhs4_1247x', 'feeder_p1rdt1011-p1rhs5_1247x']"
    }
}
{
    "ACCDF54C-C90C-4C41-9D7A-5E5D3D17A4E5": {
        "mRID": "ACCDF54C-C90C-4C41-9D7A-5E5D3D17A4E5",
        "name": "feeder_p10udt4214-p10uhs4_1247x",
        "Location": "0100B0B0-C53C-484E-8110-A9340573AD8C",
        "ConnectivityNodes": "['840290C1-65BE-47CD-89AF-92A22DCDE166', '841D96A5-0231-444A-A794-DE4FA837D959', '842DB925-72AC-4C19-A4FF-725B4233085F', '8430D81C-05B4-4DC5-A730-CF050224AC2D', '84326C11-CD25-47C0-8DC1-750117C38788', '8444FE2C-F137-4A83-A248-F7290A02469E', '84517DFA-11D1-43CA-B6E5-7ACAB0085348', '8466498D-9450-4CC4-83FF-C4FF2BBEAF5F', '8477EAF6-9ED2-4D22-BCB8-0D085C3AB537', '849187F9-53DD-4826-8A9C-EE069A1DB857', '849C75F9-DB04-427D-B8F3-AD82E4007C0E', '84B4AFC1-65FC-4D4F-A793-3773D7B3DDF6', '84CBDD48-3F8B-4A28-958E-A

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [17]:
utils.write_xml(main_transfer.network, '../test_output/combined_sub.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [18]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=793ms, elapsed=792ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=811ms, commitTime=1709769822921, mutationCount=262852</p\n></html\n>'

In [19]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/p10uhs4.xml')
bg_loader.upload_from_file(filename='../test_models/p1rhs5.xml')
bg_loader.upload_from_file(filename='../test_output/combined_sub.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 65
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="240494" milliseconds="856"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 64
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="15103" milliseconds="186"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="745" milliseconds="67"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [20]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [21]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "46d3dd5d-69ee-4715-a520-f659348f71b8": {
        "mRID": "46d3dd5d-69ee-4715-a520-f659348f71b8",
        "name": "combined_sub",
        "ConnectivityNodes": "['81ee696f-15e7-4645-a252-a225bae76576', '827412a9-90c4-46fd-b332-1f0318c6aab3', '948d5349-8a5a-465e-b147-8e5295eb0dfc', 'a1e965f9-a452-4da8-95fd-dd8bde4c8768', 'bcaf3d6a-a584-49ef-b745-724985341f69', 'c050aabb-7b53-47ce-b6a7-a9cb7f3dee2d', 'd241d801-c691-4b0f-b26c-ff1d3816bfca', 'ea9f2998-b782-4e9e-a248-bb0dea9a0c90', 'eda8c11c-0e53-4494-8a56-8e7b89e6ec3f', 'f05de8d6-5d7d-4032-8997-db58f4484136', 'f3bf94e2-2559-4b55-b572-ce127f8a2b0a', 'fef4a95a-9627-476d-a9aa-5caa1f83e706', '16471886-5532-4258-b3e7-2868470aa0c7', '1ed97c50-8740-403a-adb0-04189278351d', '267ba858-9c8b-4212-8004-61c392a79925', '27e884b8-dc60-42c8-8d37-3d98018f2ef4', '2a605d3d-42be-48d9-9009-b270e79e5210', '354bae03-02cf-4b20-87a2-bed99d4b06b2', '481cf7e7-cf3a-48c6-8548-157144ddf509', '4c5cbb5d-e0f7-44d6-ba94-b66380619eda', '5552a126-d8ae-40be-990c-c0756286

In [22]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "ACCDF54C-C90C-4C41-9D7A-5E5D3D17A4E5": {
        "mRID": "ACCDF54C-C90C-4C41-9D7A-5E5D3D17A4E5",
        "name": "feeder_p10udt4214-p10uhs4_1247x",
        "Location": "0100B0B0-C53C-484E-8110-A9340573AD8C",
        "ConnectivityNodes": "['840290C1-65BE-47CD-89AF-92A22DCDE166', '841D96A5-0231-444A-A794-DE4FA837D959', '842DB925-72AC-4C19-A4FF-725B4233085F', '8430D81C-05B4-4DC5-A730-CF050224AC2D', '84326C11-CD25-47C0-8DC1-750117C38788', '8444FE2C-F137-4A83-A248-F7290A02469E', '84517DFA-11D1-43CA-B6E5-7ACAB0085348', '8466498D-9450-4CC4-83FF-C4FF2BBEAF5F', '8477EAF6-9ED2-4D22-BCB8-0D085C3AB537', '849187F9-53DD-4826-8A9C-EE069A1DB857', '849C75F9-DB04-427D-B8F3-AD82E4007C0E', '84B4AFC1-65FC-4D4F-A793-3773D7B3DDF6', '84CBDD48-3F8B-4A28-958E-A9FA3755EBCB', '84DB9823-18D3-41FC-B669-6FB73BA21A54', '84DFE86F-B5B4-4CF5-9E63-57737FAF1C8E', '84E4C561-88CC-4CD6-9699-C05B9E74687B', '85003D54-B9AD-4745-B897-0309A9CFF2AE', '850D28A7-2B99-4F69-8F32-E6489A4A2E07', '8580BF58-330F-4D66-8B2D-E93EC9721

In [23]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 3654.5263039998945 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [24]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/combined_and_feeders.xml')